## Méthodes d'optimisation stochastique

## I. Minimisation stochastique d'une fonction déterministe


On s'intéresse au problème
$$\min f(x) =\max_{i=1\dots m}|a_i^Tx-b_i |.$$
On suppose que les $a_i$ sont les lignes $i$ d'une matrice $A$ de taille $100 \times 20$ , et les $b_i$ sont les lignes d'un second membre $b$. Ces quantités sont générées une fois pour toutes à partir de distributions Gaussiennes de moyenne nulle et d'écart type identité.

**Question 1 :** Constuire $A$ et $b$. Proposer le calcul d'un sous-gradient en $x$ de $f$.


In [1]:
# Construction des données A  et b
# Insérer votre code
A = randn(100,20);
b = randn(100,1);
x = ones(20,1);
# Fin insérer code

# Fonction calculant un sous-gradient en x de f
function subgrad(A,b,x) 
    # Insérer votre code
    C = A*x - b;
    
    f,index = findmax(abs.(C));
    i0 = index[1];
    ssgrad = A[i0,:]*sign(C[i0]);
    
    return ssgrad;
    # Fin insérer code
end
subgrad(A,b,x);

**Question 2 :** Ce problème peut se reformuler comme un problème de programmation linéaire : $$(\mathcal{P}_{lp})\quad \left\{ \begin{array}{c} \min_{(x,R)\in \mathbb{R}^n\times \mathbb{R}} h(x,R)=R\\
s.c. \quad-R*e\leq A*x-b\leq R*e\end{array}\right.$$ avec $e=[1,\cdots,1]^T\in \mathbb{R}^m$. Résoudre le problème $(\mathcal{P}_{lp})$ en utilisant le solveur "GLPK" de la librairie JuMP. Plus d'informations sont disponibles ici :  http://www.juliaopt.org/JuMP.jl/latest/quickstart/

In [6]:
using Pkg
#Pkg.add("JuMP"); 
#Pkg.add("GLPK");
using JuMP
using GLPK

# Définition du modèle
# Insérer votre code
mymodel = Model(GLPK.Optimizer);
# Fin insérer code

# Définition des variables d'optimisation
# Insérer votre code
m = 100
n = 20
@variable(mymodel, R)
@variable(mymodel, x[i=1:n])
# Fin insérer code

# Définition de la fonctionnelle à minimiser
#Insérer votre code
@objective(mymodel, Min, R);
# Fin insérer code

# Définition des contraintes
# Insérer votre code
for i=1:m
    @constraint(mymodel, sum(A[i,j]*x[j] for j=1:n) - b[i]  <= R)
    @constraint(mymodel, sum(A[i,j]*x[j] for j=1:n) - b[i]  >= -R)
end
# Fin insérer code
        
# Résolution        
# Insérer votre code
optimize!(mymodel)
termination_status(mymodel)
# Fin insérer code
        
# Résultats à optimalité                
# Insérer votre code
f_star = objective_value(mymodel);
xstar=value.(x);
Rstar=value(R);

# Fin insérer code

println("The function value at the solution is: ",Rstar, " or ",findmax(abs.(A*xstar-b)))



The function value at the solution is: 1.653981455278875 or (1.6539814552788765, CartesianIndex(96, 1))


**Question 3 :** Résoudre le problème en utilisant un algorithme de sous-gradient. Dans un premier temps vous utiliserez un sous-gradient exact (Question 1), puis vous introduirez un bruit artificiel qui suit une distribution normale de moyenne nulle et d'écart-type $3 e-1$.

**Question 4 :** Vous afficherez les courbes de convergence de $f_{best}^k-f_{star}$, avec $f_{star}$ obtenue à la Question 1. 

In [7]:
#Pkg.add("Plots")
using Plots

#Initialisation
x = zeros(n,1);
xp = zeros(n,1);

fbest =1e10; # $f_{best}^0$: cas du sous-gradient exact
fbestp=1e10; # $f_{best}^0$: cas du sous-gradient bruité
histo =[];# Suite des itérés f_{best}^k-f_{star} pour le cas du sous-gradient exact
histop=[];# Suite des itérés f_{best}^k-f_{star}, pour le cas du sous-gradient bruité

#Niveau de bruit
noise_lvl=.3;

# Resolution 
# Insérer votre code

i = 1;
while i <= m
    choix_pas = (1e-4)/sqrt(i);
    g=subgrad(A,b,x);
    x = x - choix_pas*g;
    fbest_1,~ = findmax(abs.(A*x-b));  #fval
    fbest  = min(fbest,fbest_1);
    append!(histo, fbest-f_star);
    i = i + 1;
end

i = 1;
#ajout du bruit artificiel 
xp = x + noise_lvl*rand(n,1);
while i <= m
    choix_pas = (1e-4)/sqrt(i);
    g = subgrad(A,b,xp);
    xp = xp - choix_pas*g;
    fbest_2,~ = findmax(abs.(A*xp-b));
    fbestp = min(fbestp,fbest_2);
    append!(histop, fbestp-f_star);
    i = i + 1;
end


# Fin insérer code

#Affichage des courbes de convergence
plotly();
iter=1:100;
hf=[histo,histop];
plot(iter,hf,title="Convergence curves",label=["Exact" "Noisy"],lw=3)

<!DOCTYPE html>
 
 
 Plots.jl

## II. Minimisation stochastique d'une fonction stochatique



On s'intéresse au problème
$$\min_x f(x) =\text{E} (\max_{i=1\dots m}|a_i^Tx-b_i |).$$
On suppose que les $a_i$ sont les lignes $i$ d'une matrice $A$ de taille $100 \times 20$ , et les $b_i$ sont les lignes d'un second membre $b$. Ces quantités sont générées une fois pour toutes à partir de distributions Gaussiennes de moyenne connue $\bar{A}$ et $\bar{b}$ (non nécesairement nulle) et d'écart type identité.

**Question 5 :** Proposer deux fonctions d'évaluation de la fonction $f$ et d'un sous-gradient de $f$ basées sur des échantillons de taille $M$.


In [8]:
# Evaluation de f
function fvals(Abar,bbar,noise,xs,M) 
    # Abar, bbar : moyenne des données
    # noise : niveau de bruit 
    # xs : vecteur courant 
    # M: taille de l'échantillon

# Insérer votre code
   ech = zeros(M);
   m = 100;
   n = 20;
   k=1;
    
   while k <= M        
       b = bbar + noise*rand(m,1);
       A = Abar + noise*rand(m,n);
       ech[k],~ = findmax(abs.(A*xs-b));        
       k=k+1;
    end
   return (1/M)sum(ech)
    
# Fin insérer code
end

# Evaluation d'un sous-gradient
function subgrads(Abar,bbar,noise,xs,M)
    # Abar, bbar : moyenne des données
    # noise : niveau de bruit 
    # xs : vecteur courant 
    # M: taille de l'échantillon
    
# Insérer votre code
    m = 100;
    n = 20;
    ech=zeros(n,M);
    k=1;
    
    while k <= M       
        b=bbar+noise*rand(m,1);
        A=Abar+noise*rand(m,n);       
        ech[:,k]= subgrad(A,b,xs);
        k=k+1;
    end
    return (1/M)*sum(ech, dims=2) 
# Fin insérer code

end

fvals(ones(100,20),20*ones(100,1),0.1,ones(20,1),500) 
subgrads(ones(100,20),20*ones(100,1),0.1,ones(20,1),5)



20×1 Array{Float64,2}:
 1.0449763239653438
 1.0862675903505683
 1.0627288741718424
 1.0828854690936056
 1.0840781602784078
 1.0403113739655023
 1.064819661322027 
 1.067304437006637 
 1.077780078580677 
 1.0768270161677236
 1.0646839249126305
 1.0674658287926504
 1.0562267041382032
 1.0632677159133448
 1.0518007210368314
 1.0575149903199372
 1.0621544703340795
 1.0654714900452908
 1.0575513392053606
 1.0297794451054283

**Question 6 :** Comparer les courbes de convergence du problème déterministe $$ \min_x f(x) = \max_{i=1\dots m}|\text{E} (a_i)^Tx-\text{E} (b_i) |,$$ et du problème stochastique obtenu avec $M=10,100,1000$ échantillons.

In [9]:
# Données
Abar=10*ones(m,n)+1*randn(m,n);
bbar=10*randn(m,1);

#x_0
xd = zeros(n,1); # problème déterministe
xs = xd; # problème stochastique

# Bruit et echantillon
M  = 200;
noise  = 4;

fbestd =1e10; # $f_{best}^0$: cas d'une résolution déterministe
fbests =1e10; # $f_{best}^0$: cas d'une résolution stochastique
histod =[]; # Suite des itérés f_{best}^k pour le cas d'une résolution déterministe
histos =[]; # Suite des itérés f_{best}^k pour le cas d'une résolution stochastique

# Insérer votre code
# Une résolution déterministe
i = 1;
while i <= 100
    choix_pas = (1e-3)/sqrt(i);
    g = subgrad(Abar,bbar,xd);
    xd = xd - choix_pas*g;
    fbest,~ = findmax(abs.(Abar*xd-bbar));
    fbestd  = min(fbestd,fbest);
    append!(histod, fbestd - f_star);
    i = i + 1;
end
#Une résolution stochastique
i = 1;
while i <= 100
    choix_pas = (1e-3)/sqrt(i);
    g = subgrads(Abar,bbar,noise,xs,M);
    xs = xs - choix_pas*g;
    fbests = fvals(Abar,bbar,noise,xs,M);
    #fbests = min(fbests,fbest);
    append!(histos, fbests-f_star);
    i = i + 1;
end
# Fin insérer code
#Affichage
#plotly();
iter=1:100;
hf=[histod,histos];
plot(iter,hf,title="Convergence curves",label=["Deterministic" "Stochastic"],lw=3)


<!DOCTYPE html>
 
 
 Plots.jl

**Question 7 :** Répéter les expériences et comparer les valeurs meilleurs valeurs de f obtenues ($f_{best}$) aprs un nombre fixé d'itérations. 

In [10]:
# Données
Abar=10*ones(m,n)+1*randn(m,n);
bbar=10*randn(m,1);

# x_0
xd = zeros(n,1); # résolution déterministe
xs = xd;  # résolution stochastique

# Bruit et echantillon
M  = 200;
noise  = 4;

#Nombre d'itérations
niter=100;

j = 0;

fbestd =1e10; # $f_{best}^0$: cas d'une résolution déterministe
fbests =1e10; # $f_{best}^0$: cas d'une résolution stochastique
fbesttd=[]; # f_{best} pour chaque expériences dans le cas d'une résolution déterministe
fbestts=[]; # f_{best} pour chaque expériences dans le cas d'une résolution stochastique

for nexp=1:20
    # Répétition des expériences    
# Insérer votre code

    # Une résolution déterministe
    i = 1;
    while i <= 100
        choix_pas = (1e-3)/sqrt(i);
        g = subgrad(Abar,bbar,xd);
        xd = xd - choix_pas*g;
        fbest,~ = findmax(abs.(Abar*xd-bbar));
        fbestd  = min(fbestd,fbest);
        #append!(fbesttd[nexp], fbestd - f_star);
        i = i + 1;
    end
    append!(fbesttd, fbestd-f_star);
       
    #Une résolution stochastique
    i = 1;
    while i <= 100
        choix_pas = (1e-3)/sqrt(i);
        g = subgrads(Abar,bbar,noise,xs,M);
        xs = xs - choix_pas*g;
        fbests = fvals(Abar,bbar,noise,xs,M);
        #fbests = min(fbests,fbest);
        #append!(fbestts[nexp], fbests-f_star);;
        i = i + 1;
    end
    append!(fbestts, fbests-f_star);
       
    
# Fin insérer code
end

#Affichage
#plotly();
iter=1:20;
hf=[fbesttd,fbestts];
plot(iter,hf,title="Convergence curves",label=["Deterministic" "Stochastic"],lw=3)

<!DOCTYPE html>
 
 
 Plots.jl